In [1]:
%%capture
# hide output for this cell
# install import library for brown_robinson_method import from another notebook
!pip install ipynb

In [11]:
import numpy as np

from ipynb.fs.defs.lab1 import brown_robinson_method, analytic_solve

In [ ]:
def get_first_saddle_point(arr):
    """
    saddle point = min in row && max in column
    :param arr: numpy array
    :return: i, j coordinates of saddle point or None
    """
    max_col = np.amax(arr, axis=0)
    min_row = np.amin(arr, axis=1)
    for i in range(len(min_row)):
        for j in range(len(max_col)):
            if max_col[j] == min_row[i]:
                return i, j
    return None


def get_closest_value(arr, h):
    """
    Return closest value from arr to h
    :param arr: numpy array
    :param h: value for nearest look
    :return: nearest value from arr
    """
    # matrix need to be flatten like array
    array = arr.flatten()
    idx = (np.abs(array - h)).argmin()
    return array[idx]

In [4]:
class ConvexConcaveGame:
    def __init__(self, a, b, c, d, e):
        """
        If not convex-concave game raise exception
        H(x, y) = ax^2 + by^2 + cxy + dx + ey
        """
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.e = e
        if a >= 0 or b <= 0:
            raise Exception("Game is not convex-concave")


    def kernel(self, x, y):
        """
        H(x, y) = ax^2 + by^2 + cxy + dx + ey
        :param x:
        :param y:
        :return:
        """
        return self.a * x ** 2 + self.b * y ** 2 + self.c * x * y + self.d * x + self.e * y

    def get_matrix(self, n):
        """
        Generate matrix for N step
        :param n: 
        :return: 
        """
        cell_function = lambda i, j: self.kernel(i / n, j / n)
        return np.fromfunction(cell_function, (n + 1, n + 1))
    
    @staticmethod
    def get_h(arr, x, y):
        return sum(arr[i, j] * x_i * y_j for i, x_i in enumerate(x) for j, y_j in enumerate(y))

    def solve(self, precision=0.1, precision_counter=5, verbose=False):
        """
        Solve convex-concave antagonistic game
        :param precision: 
        :param precision_counter: steps to stop with precision
        :param verbose: print output to stdout
        :return: 
        """
        n = 2
        precision_counter_cur = 1
        h = None

        while precision_counter_cur < precision_counter:
            matr = self.get_matrix(n)

            saddle_coords = get_first_saddle_point(matr)
            if saddle_coords:
                new_h = matr[saddle_coords]
            else:
                df, x, y, v = brown_robinson_method(matr, precision)
                new_h = get_closest_value(matr, Game.get_h(matr, x, y))

            h_coords = np.where(matr == new_h)
            x = h_coords[0] / n
            y = h_coords[1] / n

            n += 1
            if h and abs(new_h - h) < precision:
                precision_counter_cur += 1
            else:
                precision_counter_cur = 1
            h = new_h

            if verbose:
                # verbose output
                print(f"N = {n - 1}")
                print("Matrix:")
                print(matr)
                if saddle_coords:
                    print("Saddle point exists")
                else:
                    print("Saddle point not exists. Solve by Brown Robinson method.")
                print(f"x*: {x}")
                print(f"y*: {y}")
                print(f"H*: {new_h}")
                print()
        if verbose:
            print(f"Step count: {n - 2}")
            print(f"x*: {x}")
            print(f"y*: {y}")
            print(f"H*: {new_h}")
        
        return x, y, new_h
        

In [50]:
g = Game(-15, 9/2, 24, -36/5, -84/5)
g.solve(verbose=True)

N = 2
Matrix:
[[  0.     -7.275 -12.3  ]
 [ -7.35   -8.625  -7.65 ]
 [-22.2   -17.475 -10.5  ]]
Saddle point not exists. Solve by Brown Robinson method.
x*: [0.5]
y*: [0.5]
H*: -8.625

N = 3
Matrix:
[[  0.          -5.1         -9.2        -12.3       ]
 [ -4.06666667  -6.5         -7.93333333  -8.36666667]
 [-11.46666667 -11.23333333 -10.          -7.76666667]
 [-22.2        -19.3        -15.4        -10.5       ]]
Saddle point not exists. Solve by Brown Robinson method.
x*: [0.]
y*: [0.66666667]
H*: -9.2

N = 4
Matrix:
[[  0.       -3.91875  -7.275   -10.06875 -12.3    ]
 [ -2.7375   -5.15625  -7.0125   -8.30625  -9.0375 ]
 [ -7.35     -8.26875  -8.625    -8.41875  -7.65   ]
 [-13.8375  -13.25625 -12.1125  -10.40625  -8.1375 ]
 [-22.2     -20.11875 -17.475   -14.26875 -10.5    ]]
Saddle point not exists. Solve by Brown Robinson method.
x*: [0.5]
y*: [0.75]
H*: -8.418750000000001

N = 5
Matrix:
[[  0.    -3.18  -6.    -8.46 -10.56 -12.3 ]
 [ -2.04  -4.26  -6.12  -7.62  -8.76  -9.54]
 

N = 12
Matrix:
[[  0.          -1.36875     -2.675       -3.91875     -5.1
   -6.21875     -7.275       -8.26875     -9.2        -10.06875
  -10.875      -11.61875    -12.3       ]
 [ -0.70416667  -1.90625     -3.04583333  -4.12291667  -5.1375
   -6.08958333  -6.97916667  -7.80625     -8.57083333  -9.27291667
   -9.9125     -10.48958333 -11.00416667]
 [ -1.61666667  -2.65208333  -3.625       -4.53541667  -5.38333333
   -6.16875     -6.89166667  -7.55208333  -8.15        -8.68541667
   -9.15833333  -9.56875     -9.91666667]
 [ -2.7375      -3.60625     -4.4125      -5.15625     -5.8375
   -6.45625     -7.0125      -7.50625     -7.9375      -8.30625
   -8.6125      -8.85625     -9.0375    ]
 [ -4.06666667  -4.76875     -5.40833333  -5.98541667  -6.5
   -6.95208333  -7.34166667  -7.66875     -7.93333333  -8.13541667
   -8.275       -8.35208333  -8.36666667]
 [ -5.60416667  -6.13958333  -6.6125      -7.02291667  -7.37083333
   -7.65625     -7.87916667  -8.03958333  -8.1375      -8.17291667

N = 16
Matrix:
[[  0.          -1.03242188  -2.0296875   -2.99179688  -3.91875
   -4.81054688  -5.6671875   -6.48867188  -7.275       -8.02617188
   -8.7421875   -9.42304688 -10.06875    -10.67929688 -11.2546875
  -11.79492188 -12.3       ]
 [ -0.50859375  -1.44726562  -2.35078125  -3.21914063  -4.05234375
   -4.85039063  -5.61328125  -6.34101563  -7.03359375  -7.69101563
   -8.31328125  -8.90039062  -9.45234375  -9.96914063 -10.45078125
  -10.89726562 -11.30859375]
 [ -1.134375    -1.97929688  -2.7890625   -3.56367188  -4.303125
   -5.00742188  -5.6765625   -6.31054688  -6.909375    -7.47304688
   -8.0015625   -8.49492188  -8.953125    -9.37617188  -9.7640625
  -10.11679688 -10.434375  ]
 [ -1.87734375  -2.62851563  -3.34453125  -4.02539062  -4.67109375
   -5.28164062  -5.85703125  -6.39726563  -6.90234375  -7.37226563
   -7.80703125  -8.20664063  -8.57109375  -8.90039062  -9.19453125
   -9.45351562  -9.67734375]
 [ -2.7375      -3.39492187  -4.0171875   -4.60429688  -5.15625
   -5.67